In [1]:
from copy import deepcopy
from pathlib import Path

import bw2data
import pandas as pd

from enbios import Experiment, report
from enbios.models.models import ExperimentData

In this demo, we demonstrate the basic working of Enbios. There are 4 main parts to this introduction. The first three parts, incrementally extend the experiment, while the 4. part displays some useful functionalities, that helps to understand how to configure enbios.

For this introduction, we are going to use Brightway to do LCA calculations of ecoinvent (v.3.9.1) activities, that are at the bottom of the MuSIASEM hierarchy, so one should have a brigthway project, with some the ecoinvent dataset created before starting.

In the first step, we get some brighway activities are construct the enbios configuration step by step.

In [2]:
# get an overview of brightway projects and databases
report()

Project: default
[]
Project: ecoinvent_391
['biosphere3', 'ecoinvent_391_cutoff']
Project: Spain case study
['biosphere', 'food']
Project: supply chain graph
['bike']
Project: matrix-test
['db']
Project: nonlinear-method-test
['db']
Project: ecoinvent_391_apos
['biosphere3', 'ecoinvent_391_apos']
Project: 1
[]
Project: ecoinvent_391_consequential
['biosphere3', 'ecoinvent_391_consequential']
Project: super_simple_builder_test
['db']
Project: builder_test
['db']
Project: ppp
['db']
Project: 
[]
Project: seed_import
['biosphere3', 'ecoi39co', 'batteries', 'PV', 'cobalt']
Project: bw_temporalis example
['example']
Project: lci-model
[]
Project: test-del
['db']
Project: lci_model
['biosphere3', 'cutoff391', 'new_db']
Project: LIVENreg_ei_391_con
['biosphere3', 'ei391cutoff']


In [3]:
# select the brightway project and database (e.g. some ecoinvent database)
PROJECT_NAME = "ecoinvent_391"
DATABASE = "ecoinvent_391_cutoff"

bw2data.projects.set_current(PROJECT_NAME)
db = bw2data.Database(DATABASE)

# Simple example experiment
Let's get 2 wind turbines from Spain

In [4]:
wind_turbines_spain = db.search(
    "electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"}
)[:2]
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, ES, None)]

In [5]:
# Now we use those, to define 2 leaf-nodes in our hierarchy.
experiment_nodes = []

for activity in wind_turbines_spain:
    experiment_nodes.append(
        {
            "name": activity["name"],
            "adapter": "brightway-adapter",
            "config": {"code": activity["code"]},
        }
    )

In [6]:
# we can modify the output of the activities, by default it is the reference product (1 of the activity unit)
experiment_nodes[0]["config"]["default_output"] = {
    "unit": "kilowatt_hour",
    "magnitude": 3,
}
experiment_nodes

[{'name': 'electricity production, wind, 1-3MW turbine, onshore',
  'adapter': 'brightway-adapter',
  'config': {'code': 'ed3da88fc23311ee183e9ffd376de89b',
   'default_output': {'unit': 'kilowatt_hour', 'magnitude': 3}}},
 {'name': 'electricity production, wind, 1-3MW turbine, offshore',
  'adapter': 'brightway-adapter',
  'config': {'code': '6ebfe52dc3ef5b4d35bb603b03559023'}}]

In [7]:
hierarchy = {
    "name": "root",
    "aggregator": "sum-aggregator",
    "children": experiment_nodes,
}

In [8]:
# select 2 random methods and convert them into the form for enbios2
experiment_methods = [bw2data.methods.random() for _ in range(2)]

experiment_methods = {m[-1]: m for m in experiment_methods}
experiment_methods

{'hazardous waste no LT': ('EDIP 2003 no LT',
  'land filling no LT',
  'hazardous waste no LT'),
 'photochemical oxidant formation potential: ecosystems (EOFP)': ('ReCiPe 2016 v1.03, midpoint (H)',
  'photochemical oxidant formation: terrestrial ecosystems',
  'photochemical oxidant formation potential: ecosystems (EOFP)')}

In [9]:
# alternatively, we could just specify two methods
experiment_methods = {
    "GWP1000": (
        "ReCiPe 2016 v1.03, midpoint (H)",
        "climate change",
        "global warming potential (GWP1000)",
    ),
    "LOP": (
        "ReCiPe 2016 v1.03, midpoint (E)",
        "land use",
        "agricultural land occupation (LOP)",
    ),
    "WCP": (
        "ReCiPe 2016 v1.03, midpoint (E)",
        "water use",
        "water consumption potential (WCP)",
    ),
}

In [10]:
# let's store the raw data, because we want to modify it later
simple_raw_data = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            "config": {"bw_project": PROJECT_NAME},
            "methods": experiment_methods,
        }
    ],
    "hierarchy": hierarchy,
}

# make a first validation of the experiment data
exp_data = ExperimentData(**simple_raw_data)
exp_data.model_dump(exclude_unset=True)

{'adapters': [{'adapter_name': 'brightway-adapter',
   'config': {'bw_project': 'ecoinvent_391'},
   'methods': {'GWP1000': ('ReCiPe 2016 v1.03, midpoint (H)',
     'climate change',
     'global warming potential (GWP1000)'),
    'LOP': ('ReCiPe 2016 v1.03, midpoint (E)',
     'land use',
     'agricultural land occupation (LOP)'),
    'WCP': ('ReCiPe 2016 v1.03, midpoint (E)',
     'water use',
     'water consumption potential (WCP)')}}],
 'hierarchy': {'name': 'root',
  'aggregator': 'sum-aggregator',
  'children': [{'name': 'electricity production, wind, 1-3MW turbine, onshore',
    'config': {'code': 'ed3da88fc23311ee183e9ffd376de89b',
     'default_output': {'unit': 'kilowatt_hour', 'magnitude': 3}},
    'adapter': 'brightway-adapter'},
   {'name': 'electricity production, wind, 1-3MW turbine, offshore',
    'config': {'code': '6ebfe52dc3ef5b4d35bb603b03559023'},
    'adapter': 'brightway-adapter'}]}}

In [11]:
# create experiment object. This will validate the activities, their outputs, the methods and the scenarios.
simple_experiment: Experiment = Experiment(simple_raw_data)

In [12]:
print(simple_experiment.info())

Experiment: 
Structural nodes: 2
root - sum-aggregator
 electricity production, wind, 1-3MW turbine, onshore - brightway-adapter
 electricity production, wind, 1-3MW turbine, offshore - brightway-adapter
Methods: 3
 bw.GWP1000
 bw.LOP
 bw.WCP
Hierarchy (depth): 2
Scenarios: 1



## Running the experiment

In [13]:
# run all scenarios at once, Results will be returned as dictionary
# <scenario_name : str> : <scenario_result : dict>
# <scenario_result> : dict represents the root node of the results tree, with the following keys:
# name, output, results, children
# where results, is a dictionary of impacts to dict: unit, magnitude | multi_magnitude
# The first and only positional parameter 'results_as_dict' can be set to False, to get the tree unserialized as :  BasicTreeNode[ScenarioResultNodeData]
results = simple_experiment.run()

2024-04-22 09:22:24,994 - ............demos.enbios.base - INFO - Running scenario 'default scenario'


# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a `data` object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result`.

In [14]:
results

{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq',
    'magnitude': 0.06275128228698862},
   'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0035793235538536016},
   'WCP': {'unit': 'cubic meter', 'magnitude': 0.0005971209633515382}},
  'output': [{'unit': 'kilowatt_hour', 'magnitude': 4.0}],
  'children': [{'name': 'electricity production, wind, 1-3MW turbine, onshore',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'magnitude': 0.04573083403096655},
     'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0031983593461469605},
     'WCP': {'unit': 'cubic meter', 'magnitude': 0.0004090798127341177}},
    'output': [{'unit': 'kilowatt_hour', 'magnitude': 3.0}]},
   {'name': 'electricity production, wind, 1-3MW turbine, offshore',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'magnitude': 0.01702044825602208},
     'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0003809642077066412},
     'WCP': {'unit': 'cubic meter', 'magnitude': 0.000188041

In [15]:
# we can dump the results into a csv file
simple_experiment.results_to_csv("test.csv")
pd.read_csv("test.csv").fillna("")

,scenario,lvl_0,lvl_1,GWP1000_magnitude (kg CO2-Eq),LOP_magnitude (m2*a crop-Eq),WCP_magnitude (cubic meter),output
0,default scenario,root,,0.062751,0.003579,0.000597,"[{'unit': 'kilowatt_hour', 'magnitude': 4.0}]"
1,,,"electricity production, wind, 1-3MW turbine, o...",0.045731,0.003198,0.000409,"[{'unit': 'kilowatt_hour', 'magnitude': 3.0}]"
2,,,"electricity production, wind, 1-3MW turbine, o...",0.017020,0.000381,0.000188,"[{'unit': 'kilowatt_hour', 'magnitude': 1.0}]"


In [16]:
simple_experiment.scenarios[0].result_to_dict()

{'name': 'root',
 'results': {'GWP1000': {'unit': 'kg CO2-Eq',
   'magnitude': 0.06275128225929673},
  'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0035793235283765198},
  'WCP': {'unit': 'cubic meter', 'magnitude': 0.0005971209632048599}},
 'output': {'unit': 'kilowatt_hour', 'magnitude': 4.0},
 'children': [{'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'results': {'GWP1000': {'unit': 'kg CO2-Eq',
     'magnitude': 0.045730834001617954},
    'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.003198359333504506},
    'WCP': {'unit': 'cubic meter', 'magnitude': 0.0004090798125724361}},
   'output': {'unit': 'kilowatt_hour', 'magnitude': 3.0}},
  {'name': 'electricity production, wind, 1-3MW turbine, offshore',
   'results': {'GWP1000': {'unit': 'kg CO2-Eq',
     'magnitude': 0.017020448257678778},
    'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.00038096419487201346},
    'WCP': {'unit': 'cubic meter', 'magnitude': 0.00018804115063242375}},
   'output': {'unit': 'k

## Add a technology hierarchy (dendrogram) 
Let's now add a few more activities to the experiment and create a hierarchy of activities.

In [13]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

Excluding 465 filtered results


['electricity production, solar tower power plant, 20 MW' (kilowatt hour, ES, None),
 'electricity production, solar thermal parabolic trough, 50 MW' (kilowatt hour, ES, None)]

In [14]:
experiment_nodes_solar = []
for activity in solar_spain:
    experiment_nodes_solar.append(
        {"name": activity["name"], "adapter": "bw", "config": {"code": activity["code"]}}
    )

hierarchy_raw_data = deepcopy(simple_raw_data)

hierarchy_raw_data["hierarchy"] = {
    "name": "root",
    "aggregator": "sum",
    "children": [
        {"name": "wind", "aggregator": "sum", "children": experiment_nodes},
        {"name": "solar", "aggregator": "sum", "children": experiment_nodes_solar},
    ],
}

In [15]:
hierarchy_experiment: Experiment = Experiment(hierarchy_raw_data)

# Run the 2nd experiment

In [16]:
hierarchy_experiment.run()

2024-04-22 09:44:12,504 - ............demos.enbios.base - INFO - Running scenario 'default scenario'


{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq',
    'magnitude': 0.16515205296612143},
   'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.016808962919060966},
   'WCP': {'unit': 'cubic meter', 'magnitude': 0.0010633319876832354}},
  'output': [{'unit': 'kilowatt_hour', 'magnitude': 6.0}],
  'children': [{'name': 'wind',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'magnitude': 0.06275128228698862},
     'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0035793235538536016},
     'WCP': {'unit': 'cubic meter', 'magnitude': 0.0005971209633515382}},
    'output': [{'unit': 'kilowatt_hour', 'magnitude': 4.0}],
    'children': [{'name': 'electricity production, wind, 1-3MW turbine, onshore',
      'results': {'GWP1000': {'unit': 'kg CO2-Eq',
        'magnitude': 0.04573083403096655},
       'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0031983593461469605},
       'WCP': {'unit': 'cubic meter', 'magnitude': 0.0004090798127341177}},
      'output':

In [17]:
# optional storing, to use it in other notebooks
# import pickle
# 
# pickle.dump(hierarchy_experiment,open("exp.pickle", "bw"))

In [21]:
# print(json.dumps((exp.scenarios[0].result_to_dict()), indent=2))
hierarchy_experiment.scenarios[0].results_to_csv(
    "test.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("test.csv").fillna("")

,root,technology,activity,GWP1000_magnitude (kg CO2-Eq),LOP_magnitude (m2*a crop-Eq),WCP_magnitude (cubic meter),output_unit,output_magnitude
0,root,,,0.165152,0.016809,0.001063,kilowatt_hour,6.0
1,,wind,,0.062751,0.003579,0.000597,kilowatt_hour,4.0
2,,,"electricity production, wind, 1-3MW turbine, o...",0.045731,0.003198,0.000409,kilowatt_hour,3.0
3,,,"electricity production, wind, 1-3MW turbine, o...",0.017020,0.000381,0.000188,kilowatt_hour,1.0
4,,solar,,0.102401,0.013230,0.000466,kilowatt_hour,2.0
5,,,"electricity production, solar tower power plan...",0.048205,0.006941,0.000211,kilowatt_hour,1.0
6,,,"electricity production, solar thermal paraboli...",0.054196,0.006289,0.000255,kilowatt_hour,1.0


## Create several scenarios

In [22]:
from typing import Optional
from random import randint


def create_normal_scenario():
    return {
        "name": "normal scenario",
        "nodes": {
            act: ("kilowatt_hour", 1)
            for act in hierarchy_experiment.structural_nodes_names
        },
    }


def create_random_scenario(scneario_name: Optional[str] = None):
    return {
        "name": scneario_name,
        "nodes": {
            act: ("kilowatt_hour", randint(1, 10))
            for act in hierarchy_experiment.structural_nodes_names
        },
    }


scenarios_raw_data = deepcopy(hierarchy_raw_data)
scenarios_raw_data["scenarios"] = [
    create_normal_scenario(),
    create_random_scenario(),
    create_random_scenario(),
]

scenarios_raw_data["scenarios"]

[{'name': 'normal scenario',
  'nodes': {'electricity production, wind, 1-3MW turbine, onshore': ('kilowatt_hour',
    1),
   'electricity production, wind, 1-3MW turbine, offshore': ('kilowatt_hour',
    1),
   'electricity production, solar tower power plant, 20 MW': ('kilowatt_hour',
    1),
   'electricity production, solar thermal parabolic trough, 50 MW': ('kilowatt_hour',
    1)}},
 {'name': None,
  'nodes': {'electricity production, wind, 1-3MW turbine, onshore': ('kilowatt_hour',
    8),
   'electricity production, wind, 1-3MW turbine, offshore': ('kilowatt_hour',
    7),
   'electricity production, solar tower power plant, 20 MW': ('kilowatt_hour',
    1),
   'electricity production, solar thermal parabolic trough, 50 MW': ('kilowatt_hour',
    1)}},
 {'name': None,
  'nodes': {'electricity production, wind, 1-3MW turbine, onshore': ('kilowatt_hour',
    9),
   'electricity production, wind, 1-3MW turbine, offshore': ('kilowatt_hour',
    6),
   'electricity production, solar

In [23]:
scenarios_raw_data

{'adapters': [{'adapter_name': 'brightway-adapter',
   'config': {'bw_project': 'ecoinvent_391'},
   'methods': {'GWP1000': ('ReCiPe 2016 v1.03, midpoint (H)',
     'climate change',
     'global warming potential (GWP1000)'),
    'LOP': ('ReCiPe 2016 v1.03, midpoint (E)',
     'land use',
     'agricultural land occupation (LOP)'),
    'WCP': ('ReCiPe 2016 v1.03, midpoint (E)',
     'water use',
     'water consumption potential (WCP)')}}],
 'hierarchy': {'name': 'root',
  'aggregator': 'sum',
  'children': [{'name': 'wind',
    'aggregator': 'sum',
    'children': [{'name': 'electricity production, wind, 1-3MW turbine, onshore',
      'adapter': 'brightway-adapter',
      'config': {'code': 'ed3da88fc23311ee183e9ffd376de89b',
       'default_output': {'unit': 'kilowatt_hour', 'magnitude': 3}}},
     {'name': 'electricity production, wind, 1-3MW turbine, offshore',
      'adapter': 'brightway-adapter',
      'config': {'code': '6ebfe52dc3ef5b4d35bb603b03559023'}}]},
   {'name': 'solar

In [24]:
scenarios_experiment = Experiment(scenarios_raw_data)

 ## Run the experiment for the 3rd time
This time will likely take some more time since we need to run 2 scenarios. 

In [25]:
_ = scenarios_experiment.run()
# don't print the whole result

2024-02-07 15:04:07,080 - ............enbios.demos.enbios.base - INFO - Running scenario 'normal scenario'
2024-02-07 15:04:07,084 - ............enbios.demos.ADAPTER::(brightway-adapter) - INFO - Brightway adapter: Run distribution 1/1
2024-02-07 15:04:51,874 - ............enbios.demos.enbios.base - INFO - Running scenario 'Scenario 1'
2024-02-07 15:04:51,886 - ............enbios.demos.ADAPTER::(brightway-adapter) - INFO - Brightway adapter: Run distribution 1/1
2024-02-07 15:05:39,431 - ............enbios.demos.enbios.base - INFO - Running scenario 'Scenario 2'
2024-02-07 15:05:39,439 - ............enbios.demos.ADAPTER::(brightway-adapter) - INFO - Brightway adapter: Run distribution 1/1


In [27]:
scenarios_experiment.scenarios[0].results_to_csv(
    "s1.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("s1.csv").fillna("")
Path("s1.csv").unlink()  # delete the file again

We can also now run any new scenario configuration for the given experiment using `run_scenario_config`

In [28]:
new_random_scenario = create_random_scenario()
new_random_scenario["name"] = "new scenario"
new_results = scenarios_experiment.run_scenario_config(new_random_scenario)

2024-02-07 15:07:04,809 - ............enbios.demos.enbios.base - INFO - Running scenario 'new scenario'
2024-02-07 15:07:04,814 - ............enbios.demos.ADAPTER::(brightway-adapter) - INFO - Brightway adapter: Run distribution 1/1


## Inspecting the results

We can now do some transformations of the results. For that is useful to know how to retrieve is singular result from a scenario result. 
The result of a scenario is a tree structure, where the nodes `name`s are activity aliases or names defined in the hierarchy. With the function of BasicTreeNode.find_child_by_name we can directly access the result of a node.    

Following we transform the results into a dictionary of the following structure:
```json
{
    "node": {
        "method": "[list of results for each scenario]"
    }
}
```

In [29]:
all_results = {}
for node in scenarios_experiment.structural_nodes_names:
    all_results[node] = {method: [] for method in scenarios_experiment.method_names}
    for scenario in scenarios_experiment.scenarios:
        activity_result = scenario.result_tree.find_subnode_by_name(node)
        for method, score in activity_result.data.results.items():
            all_results[node][method].append(score.model_dump(exclude_defaults=True))

all_results

{'electricity production, wind, 1-3MW turbine, onshore': {'GWP1000': [{'unit': 'kg CO2-Eq',
    'magnitude': 0.015243611332791748},
   {'unit': 'kg CO2-Eq', 'magnitude': 0.12194889066233398},
   {'unit': 'kg CO2-Eq', 'magnitude': 0.13719250200876407}],
  'LOP': [{'unit': 'm2*a crop-Eq', 'magnitude': 0.0010661197743596256},
   {'unit': 'm2*a crop-Eq', 'magnitude': 0.008528958194877005},
   {'unit': 'm2*a crop-Eq', 'magnitude': 0.009595078053187335}],
  'WCP': [{'unit': 'cubic meter', 'magnitude': 0.000136359937516592},
   {'unit': 'cubic meter', 'magnitude': 0.001090879500132736},
   {'unit': 'cubic meter', 'magnitude': 0.0012272394377443956}]},
 'electricity production, wind, 1-3MW turbine, offshore': {'GWP1000': [{'unit': 'kg CO2-Eq',
    'magnitude': 0.017020448257678778},
   {'unit': 'kg CO2-Eq', 'magnitude': 0.11914313779678103},
   {'unit': 'kg CO2-Eq', 'magnitude': 0.1021226895590089}],
  'LOP': [{'unit': 'm2*a crop-Eq', 'magnitude': 0.00038096419487201346},
   {'unit': 'm2*a cro